## Lesson 3: Email Assistant with Semantic Memory

We previously built an email assistant that:

- Classifies incoming messages (respond, ignore, notify)
- Drafts responses
- Schedules meetings

Now, we'll add memory.

We'll give the assistant the ability to remember details from previous emails.


In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
profile = {
    "name": "John",
    "full_name": "John Doe",
    "user_profile_background": "Senior software engineer leading a team of 5 developers",
}

prompt_instructions = {
    "triage_rules": {
        "ignore": "Marketing newsletters, spam emails, mass company announcements",
        "notify": "Team member out sick, build system notifications, project status updates",
        "respond": "Direct questions from team members, meeting requests, critical bug reports",
    },
    "agent_instructions": "Use these tools when appropriate to help manage John's tasks efficiently.",
}

email = {
    "from": "Alice Smith <alice.smith@company.com>",
    "to": "John Doe <john.doe@company.com>",
    "subject": "Quick question about API documentation",
    "body": """
Hi John,

I was reviewing the API documentation for the new authentication service and noticed a few endpoints seem to be missing from the specs. Could you help clarify if this was intentional or if we should update the docs?

Specifically, I'm looking at:
- /auth/refresh
- /auth/validate

Thanks!
Alice""",
}

In [3]:
from langchain_openai.chat_models import ChatOpenAI
from pydantic import BaseModel, Field
from typing_extensions import Annotated, Literal, TypedDict

In [ ]:
llm = ChatOpenAI(
    model="gpt-4.1",
    base_url="https://models.inference.ai.azure.com/",
    temperature=0.5,
)

In [5]:
class Router(BaseModel):
    """Analyze the unread email and route it according to its content."""

    reasoning: str = Field(
        description="Step-by-step reasoning behind the classification."
    )
    classification: Literal["ignore", "respond", "notify"] = Field(
        description="The classification of an email: 'ignore' for irrelevant emails, "
        "'notify' for important information that doesn't need a response, "
        "'respond' for emails that need a reply",
    )

In [6]:
llm_router = llm.with_structured_output(Router)

In [7]:
from langchain_core.tools import tool
from prompts import triage_system_prompt, triage_user_prompt

In [8]:
@tool
def write_email(to: str, subject: str, content: str) -> str:
    """Write and send an email."""
    # Placeholder response - in real app would send email
    return f"Email sent to {to} with subject '{subject}'"


@tool
def schedule_meeting(
    attendees: list[str], subject: str, duration_minutes: int, preferred_day: str
) -> str:
    """Schedule a calendar meeting."""
    # Placeholder response - in real app would check calendar and schedule
    return f"Meeting '{subject}' scheduled for {preferred_day} with {len(attendees)} attendees"


@tool
def check_calendar_availability(day: str) -> str:
    """Check calendar availability for a given day."""
    # Placeholder response - in real app would check actual calendar
    return f"Available times on {day}: 9:00 AM, 2:00 PM, 4:00 PM"

### Define tools for managing memory


In [9]:
from langgraph.store.memory import InMemoryStore
from langmem import create_manage_memory_tool, create_search_memory_tool

In [10]:
from langchain_ollama.embeddings import OllamaEmbeddings

In [11]:
store = InMemoryStore(index={"embed": OllamaEmbeddings(model="nomic-embed-text:v1.5")})

In [12]:
manage_memory_tool = create_manage_memory_tool(
    namespace=("email_assistant", "{langgraph_user_id}", "collection")
)
search_memory_tool = create_search_memory_tool(
    namespace=("email_assistant", "{langgraph_user_id}", "collection")
)

In [13]:
print(manage_memory_tool.name)

manage_memory


In [14]:
print(manage_memory_tool.description)

Create, update, or delete a memory to persist across conversations.
Include the MEMORY ID when updating or deleting a MEMORY. Omit when creating a new MEMORY - it will be created for you.
Proactively call this tool when you:

1. Identify a new USER preference.
2. Receive an explicit USER request to remember something or otherwise alter your behavior.
3. Are working and want to record important context.
4. Identify that an existing MEMORY is incorrect or outdated.


In [15]:
manage_memory_tool.args

{'content': {'anyOf': [{'type': 'string'}, {'type': 'null'}],
  'default': None,
  'title': 'Content'},
 'action': {'default': 'create',
  'enum': ['create', 'update', 'delete'],
  'title': 'Action',
  'type': 'string'},
 'id': {'anyOf': [{'format': 'uuid', 'type': 'string'}, {'type': 'null'}],
  'default': None,
  'title': 'Id'}}

In [16]:
search_memory_tool.name

'search_memory'

In [17]:
search_memory_tool.description

'Search your long-term memories for information relevant to your current context.'

In [18]:
search_memory_tool.args

{'query': {'title': 'Query', 'type': 'string'},
 'limit': {'default': 10, 'title': 'Limit', 'type': 'integer'},
 'offset': {'default': 0, 'title': 'Offset', 'type': 'integer'},
 'filter': {'anyOf': [{'additionalProperties': True, 'type': 'object'},
   {'type': 'null'}],
  'default': None,
  'title': 'Filter'}}

In [19]:
agent_system_prompt_memory = """
< Role >
You are {full_name}'s executive assistant. You are a top-notch executive assistant who cares about {name} performing as well as possible.
</ Role >

< Tools >
You have access to the following tools to help manage {name}'s communications and schedule:

1. write_email(to, subject, content) - Send emails to specified recipients
2. schedule_meeting(attendees, subject, duration_minutes, preferred_day) - Schedule calendar meetings
3. check_calendar_availability(day) - Check available time slots for a given day
4. manage_memory - Store any relevant information about contacts, actions, discussion, etc. in memory for future reference
5. search_memory - Search for any relevant information that may have been stored in memory

You must always use manage_memory to record actions you took on important tasks.
</ Tools >

< Instructions >
{instructions}
</ Instructions >
"""

In [20]:
def create_prompt(state):
    return [
        {
            "role": "system",
            "content": agent_system_prompt_memory.format(
                instructions=prompt_instructions["agent_instructions"], **profile
            ),
        }
    ] + state["messages"]

In [21]:
from langgraph.prebuilt import create_react_agent

In [22]:
tools = [
    write_email,
    schedule_meeting,
    check_calendar_availability,
    manage_memory_tool,
    search_memory_tool,
]

response_agent = create_react_agent(
    model=llm,
    tools=tools,
    prompt=create_prompt,
    # Use this to ensure the store is passed to the agent
    store=store,
)

In [23]:
config = {"configurable": {"langgraph_user_id": "lance"}}

In [24]:
response = response_agent.invoke(
    {"messages": [{"role": "user", "content": "Jim is my friend"}]}, config=config
)

In [25]:
for m in response["messages"]:
    m.pretty_print()

================================ Human Message =================================

Jim is my friend
================================== Ai Message ==================================
Tool Calls:
  manage_memory (call_J1W2izOKci8c3VThcqrjTZzN)
 Call ID: call_J1W2izOKci8c3VThcqrjTZzN
  Args:
    content: Jim is John's friend.
    action: create
================================= Tool Message =================================
Name: manage_memory

created memory a3002439-5242-4cd7-9193-ec1383859fc1
================================== Ai Message ==================================

I've noted that Jim is your friend. If you need to remember anything specific about Jim or want to prioritize communications with him, just let me know!


In [26]:
response = response_agent.invoke(
    {"messages": [{"role": "user", "content": "who is jim?"}]}, config=config
)

NumPy not found in the current Python environment. The InMemoryStore will use a pure Python implementation for vector operations, which may significantly impact performance, especially for large datasets or frequent searches. For optimal speed and efficiency, consider installing NumPy: pip install numpy


In [27]:
for m in response["messages"]:
    m.pretty_print()

================================ Human Message =================================

who is jim?
================================== Ai Message ==================================
Tool Calls:
  search_memory (call_pKo7MvTccqBfvkE505CXQwCx)
 Call ID: call_pKo7MvTccqBfvkE505CXQwCx
  Args:
    query: jim
================================= Tool Message =================================
Name: search_memory

[{"namespace":["email_assistant","lance","collection"],"key":"a3002439-5242-4cd7-9193-ec1383859fc1","value":{"content":"Jim is John's friend."},"created_at":"2025-07-09T09:02:46.459474+00:00","updated_at":"2025-07-09T09:02:46.459476+00:00","score":0.6781540745932363}]
================================== Ai Message ==================================

Jim is John's friend. If you need more specific information about Jim or your relationship with him, please let me know!


In [28]:
store.list_namespaces()

[('email_assistant', 'lance', 'collection')]

In [29]:
store.search(("email_assistant", "lance", "collection"))

[Item(namespace=['email_assistant', 'lance', 'collection'], key='a3002439-5242-4cd7-9193-ec1383859fc1', value={'content': "Jim is John's friend."}, created_at='2025-07-09T09:02:46.459474+00:00', updated_at='2025-07-09T09:02:46.459476+00:00', score=None)]

In [30]:
store.search(("email_assistant", "lance", "collection"), query="jim")

[Item(namespace=['email_assistant', 'lance', 'collection'], key='a3002439-5242-4cd7-9193-ec1383859fc1', value={'content': "Jim is John's friend."}, created_at='2025-07-09T09:02:46.459474+00:00', updated_at='2025-07-09T09:02:46.459476+00:00', score=0.6781540745932363)]

### Create the rest of the agent


In [31]:
from langgraph.graph import add_messages

In [32]:
class State(TypedDict):
    email_input: dict
    messages: Annotated[list, add_messages]

In [ ]:
from typing import Literal

from langgraph.graph import END, START, StateGraph
from langgraph.types import Command

In [34]:
def triage_router(state: State) -> Command[Literal["response_agent", "__end__"]]:
    author = state["email_input"]["author"]
    to = state["email_input"]["to"]
    subject = state["email_input"]["subject"]
    email_thread = state["email_input"]["email_thread"]

    system_prompt = triage_system_prompt.format(
        full_name=profile["full_name"],
        name=profile["name"],
        user_profile_background=profile["user_profile_background"],
        triage_no=prompt_instructions["triage_rules"]["ignore"],
        triage_notify=prompt_instructions["triage_rules"]["notify"],
        triage_email=prompt_instructions["triage_rules"]["respond"],
        examples=None,
    )
    user_prompt = triage_user_prompt.format(
        author=author, to=to, subject=subject, email_thread=email_thread
    )
    result = llm_router.invoke(
        [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt},
        ]
    )

    if result.classification == "respond":
        print("📧 Classification: RESPOND - This email requires a response")
        goto = "response_agent"
        update = {
            "messages": [
                {
                    "role": "user",
                    "content": f"Respond to the email {state['email_input']}",
                }
            ]
        }
    elif result.classification == "ignore":
        print("🚫 Classification: IGNORE - This email can be safely ignored")
        update = None
        goto = END
    elif result.classification == "notify":
        # If real life, this would do something else
        print("🔔 Classification: NOTIFY - This email contains important information")
        update = None
        goto = END
    else:
        raise ValueError(f"Invalid classification: {result.classification}")

    return Command(goto=goto, update=update)

### Create email agent


In [35]:
email_agent = StateGraph(State)
email_agent = email_agent.add_node(triage_router)
email_agent = email_agent.add_node("response_agent", response_agent)
email_agent = email_agent.add_edge(START, "triage_router")
email_agent = email_agent.compile(store=store)

In [36]:
email_input = {
    "author": "Alice Smith <alice.smith@company.com>",
    "to": "John Doe <john.doe@company.com>",
    "subject": "Quick question about API documentation",
    "email_thread": """Hi John,

I was reviewing the API documentation for the new authentication service and noticed a few endpoints seem to be missing from the specs. Could you help clarify if this was intentional or if we should update the docs?

Specifically, I'm looking at:
- /auth/refresh
- /auth/validate

Thanks!
Alice""",
}

In [37]:
response = email_agent.invoke({"email_input": email_input}, config=config)

📧 Classification: RESPOND - This email requires a response


In [38]:
for m in response["messages"]:
    m.pretty_print()

================================ Human Message =================================

Respond to the email {'author': 'Alice Smith <alice.smith@company.com>', 'to': 'John Doe <john.doe@company.com>', 'subject': 'Quick question about API documentation', 'email_thread': "Hi John,\n\nI was reviewing the API documentation for the new authentication service and noticed a few endpoints seem to be missing from the specs. Could you help clarify if this was intentional or if we should update the docs?\n\nSpecifically, I'm looking at:\n- /auth/refresh\n- /auth/validate\n\nThanks!\nAlice"}
================================== Ai Message ==================================
Tool Calls:
  write_email (call_DMUP1tyGhgquMMi20lCtWYyP)
 Call ID: call_DMUP1tyGhgquMMi20lCtWYyP
  Args:
    to: alice.smith@company.com
    subject: Re: Quick question about API documentation
    content: Hi Alice,

Thanks for reaching out and for catching this. The /auth/refresh and /auth/validate endpoints should indeed be included

### Try a follow-up email


In [39]:
email_input = {
    "author": "Alice Smith <alice.smith@company.com>",
    "to": "John Doe <john.doe@company.com>",
    "subject": "Follow up",
    "email_thread": """Hi John,

Any update on my previous ask?""",
}

In [40]:
response = email_agent.invoke({"email_input": email_input}, config=config)

📧 Classification: RESPOND - This email requires a response


In [41]:
for m in response["messages"]:
    m.pretty_print()

================================ Human Message =================================

Respond to the email {'author': 'Alice Smith <alice.smith@company.com>', 'to': 'John Doe <john.doe@company.com>', 'subject': 'Follow up', 'email_thread': 'Hi John,\n\nAny update on my previous ask?'}
================================== Ai Message ==================================
Tool Calls:
  write_email (call_dNraNWaquWCLwyNp1gmJQscl)
 Call ID: call_dNraNWaquWCLwyNp1gmJQscl
  Args:
    to: alice.smith@company.com
    subject: Re: Follow up
    content: Hi Alice,

Thank you for your follow-up. I am looking into your previous request and will get back to you with an update as soon as possible. If you need anything specific in the meantime, please let me know.

Best regards,
John
================================= Tool Message =================================
Name: write_email

Email sent to alice.smith@company.com with subject 'Re: Follow up'
================================== Ai Message =================